<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli (and Marc Spiegelman)</td>
</table>

Much of this material was adapted from the first few chapters of [Larson and Bengzon: The Finite Element Method
Theory, Implementation,
and Applications](http://link.springer.com/book/10.1007%2F978-3-642-33287-6).

In [8]:
%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Finite Element Methods: an introduction

The Finite Element Method (FEM) provides yet another method of transforming continuous PDE's to systems of discrete linear and non-linear equations and is particularly popular in Engineering and Solid Mechanics (although the method is completely general).  Finite Elements has a rich mathematical basis in functional analysis as well a very wide range of specialized applications.  Needless to say, this subject and the accompanying literature is vast.  The purpose of these notes are just to give you a very basic introduction to some of the fundamental ideas in Finite Elements and their implementation.

## Finite Dimensional Function spaces

A key idea in Finite Elements is that we will be approximating continuous functions $f(x)$ as a linear combination of a finite number of basis functions

$$
    f_h(x) = \sum_{i=0}^N w_i \phi_i(x)
$$

where $\phi_i(x)$ are the basis functions and $w_i$ are a set of weights.  We've seen similar decompositions for spectral methods. However instead of using global basis functions such as $\sin(kx)$ or Chebyshev polynomials,  we will (for the most part) use piecewise polynomials.


### Piecewise Linear Functions in 1-D (P1)

![](./images/P1_function_annotated_fig1.1LB.png)

As an example, consider the piecewise linear "connect the dots" function illustrated above where we have decomposed an interval $I=[x_0,x_5]$ into 5 sub-intervals or "elements"  $e_i = [x_{i},x_{i+1}]$ with $i=1\ldots 4$.  Over each of these intervals, the function $v_h$ is a linear function of $x$ and each segment is continuous at the element edge.  Figure modified from Figure 1.1 of Larson and Bengzon.  

While $v_h(x)$ is a continous function that can be evaluated for any value of $x$, it should be clear that there are really only 6 degrees of freedom corresponding to the values at the nodes $\vec v = [v_0,v_1,v_2,v_3,\ldots,v_5]$.  By just changing these 6 numbers, we can generate an infinite number of piecewise linear functions on this triangulation.  We will call this finite dimensional space of functions $\cal V_h$, which is fully described by a mesh of elements, and an interpolant over each element.  



### Global Basis Functions

Given any discrete function space $\cal V_h$, we should be able to describe any function $v_h\in \cal V_h$ in terms of a set of basis functions that span the space.  In the case of a 1-D piecewise linear function space, the global basis functions are given by the "Hat Functions"  

$$
\phi_i(x) = \left\{
\begin{array}{ll}
(x - x_{i-1})/h_{i-1}, & x\in e_{i-1} \\
(x_i - x)/h_{i}, & x\in e_{i} \\
0, & \text{otherwise} \\
\end{array}
\right.
$$

Which have the important property that $\phi_i(x_j) = \delta_{ij}$.  A typical hat function is shown in Figure 1.2 from Larson and Bengson.
![](./images/P1_Hat_function_fig1.2LB.png)

While it's not entirely obvious,  we can construct all piecewise linear functions in $\cal V_h$ as a linear combination of these hat functions, i.e. 

$$
f_h(x) = \sum_{i=0}^5 w_i\phi_i(x)
$$


### Local Basis Functions

To make this a bit clearer it is worth looking at how these basis functions interpolate $v$ over a single element.

![](./images/P1LocalBases.png)

> **Figure *3*: Definition of Global and Local Basis Functions**

Consider just the portion of $v(x)$ that lies in element $e_i$.  Clearly the only global basis functions that have support in element $e_i$ are $\phi_i(x)$ and $\phi_{i+1}(x)$.  We claim that over this element
$$
v(x) = v_i\phi_i(x) + v_{i+1}\phi_{i+1}(x)\quad\text{for}\quad x\in [x_i,x_{i+1}]
$$

This is clearly true at the nodes because of the property that $\phi_i(x_j) = \delta_{ij}$.  

Between the nodes, we expect that $\phi_i$ and $\phi_{i+1}$ provide linear interpolation.  If we just consider the *local basis functions* which are the parts of $\phi_i$ and $\phi_{i+1}$ in element $e_i$

\begin{align}
  N_0(x) & = (x_{i+1} - x)/(x_{i+1} - x_i)\\
  N_1(x) & = (x - x_i)/(x_{i+1} - x_i)\\
\end{align}

it becomes clear that these are just the Lagrange Polynomials of order 1 that interpolate any linear function over the interval $[x_i,x_{i+1}]$.   Just to check note that if $x* = x_i + th_i$ where $t$ is the fractional distance across the element with lenght $h_i = x_{i+1} - x_i$, then
\begin{align}
  N_0(x*) & = (x_{i+1} - x_i -t h_i)/h_i = (1 -t)\\
  N_1(x*) & = (x_i + th_i - x_i)/h_i = t\\
\end{align}

so that within $e_i$
$$ 
   v(x) = v_i N_0(x) + v_{i+1}N_1(x) = v_i(1 - t)  + v_{i+1} t
$$
with $t\in[0,1]$

Thus our full function $v(x)$ over the full domain $\Omega = [x_0,x_5]$ is just the union of these local linear pieces.  Throughout this introduction we will often move between the global view and the element view depending on which frame of reference is more convenience.

### Higher order polynomial Function Spaces

The space of piecewise linear functions is often referred to as $P_1$ but there are other choices.  For example, the space of piecewise quadratic functions ($P_2$) has a set of local basis functions that are simply the Lagrange Polynomials of order 2 and require three degrees of freedom per node.  The global basis functions are simply constructed out of the local ones.  The Figure shows a subset of basis functions for $P_2$ which are different for degrees of freedom at the centers and edges of elements (original figure from [Introduction to Finite Element Methods](http://hplgit.github.io/INF5620/doc/pub/sphinx-fem)

![](http://hplgit.github.io/INF5620/doc/pub/sphinx-fem/_images/mpl_fe_basis_p2_4e_lab.png)



## Interpolation of a function $f(x)$ onto $\cal V$

Given a set of basis functions $\phi_i(x)$ for a function space $\cal V$, it should be clear that not all functions can be represented exactly, without error in $\cal V$.  For example, given a smooth function $f(x)$ the question arises how to construct approximations $f_h(x) \in \cal V$ that minimize some measure of the error.  Actually there are multiple approaches.  The simplest approximation is simply the *interpolation* of $f$ onto $\cal V$ which we will denote $\pi f$ and define as the function $\pi f \in \cal V$ such that

$$
    \pi f (x) = \sum_{i=0}^N f(x_i)\phi(x)
$$

Given the interpolating properties of the Lagrange polynomials, it should be clear that

$$
\pi f(x_j) = f(x_j)
$$

i.e. the two functions have zero error at the degrees of freedom. 
Figure 1.3 from Larson and Bengzon shows the linear interpolant of a function $f$ onto a single element over the interval $[x_0,x_1]$.

![](./images/f_interpolation_P1_01.png)

Figure 1.4 from Larson and Bengzon shows the linear interpolant of a function $f$ onto a piecewise linear function space $\cal V$.

![](./images/f_interpolation_P1_02.png)



### Interpolation Errors

In general there will usually be an error between $f(x)$ and its interpolant $\pi f(x)$. Quantifying this error though usually requires defining some norm and as usual there are multiple choices.  As it turns out however,  a useful norm in finite elements are the $L^2$ norm of a function over a domain $\Omega$

$$
    ||f||_{L^2} = \left[ \int_\Omega f^2 dx\right]^{1/2}
$$

and its related norm

$$
    ||f||_{L^2(I)} = \left[ \int_I f^2 dx\right]^{1/2}
$$

over a sub-interval $I$ (usually an element).

Given these definitions, and the behavior of polynomial interpolation it is relatively straightforward to put the following estimates on the interpolation error

* Interpolation error over a single P1 linear element

$$ 
|| f - \pi f||_{L^2(I)} \leq C h^2||f''||_{L^2(I)}
$$

where $I = [x_i,x{i+1}]$, $h = x_{i+1} - x{i}$, C is a constant, and $||f''||_{L^2(I)}$ is a measure of the curvature of the function over the interval.  If $h=0$ or the function is linear over the interval, then the error is zero.

* Interpolation error over the whole domain

For a piecewise polynomial function, the errors in the interpolant onto $\cal V$ are simply the summation of the errors over the element i.e. 

$$ 
|| f - \pi f||^2_{L^2} \leq C \sum_{i=0}^N h_i^4||f''||^2_{L^2(I_i)}
$$

### Convergence

Given these error estimates, it is clear that the interpolant converges to the continuous function in the $L^2$ norm as $h\rightarrow 0$ which makes sense as we expect the interpolant to be a better approximation to $f$ for finer and finer meshes. For $P_1$ functions, the error converges as $h^2$,  higher order interpolants of *smooth* functions should converge more rapidly if the higher-order piecewise polynomial is a good approximation to the continuous function $f$.



## $L_2$ Projection of a function $f(x)$ onto $\cal V$

While the interpolant $\pi f$ converges to $f$ in the $L^2$ norm as $h\rightarrow 0$, it turns out that for fixed $h$, it is not the function in $\cal V$ with the smallest $L^2$ error.  For that problem we need the orthogonal $L^2$ projection $P_h f$ which turns out to be the unique function $v_h\in\cal V$ that minimizes $||f - v_h||_{L^2}$.  
Figure 1.6 from Larson and Bengzon show the orthogonal projection of $f$ onto $\cal V$ for $f(x) = x\sin(\pi x)$

![](./images/f_projection_P1_01.png)


### Relationship to Least-Squares projection in linear algebra

Before we develop the machinery for calculating  the orthogonal $L^2$ projection of $f$ onto $\cal V$, $P_h f$,it is probably worth reviewing, briefly the related orthongal projection problem from linear algebra. 

![](./images/projection_figure.png)


Consider a 2-dimensional subspace $\cal S$ of $R^3$ spanned by two linearly independent 3-D vectors $\boldsymbol{a}_1$ and $\boldsymbol{a}_2$. i.e.

$$
\cal S = \mathrm{span}<\boldsymbol{a}_1,\boldsymbol{a}_2>
$$ 

Let $\boldsymbol{b}$ be a third vector not in $\cal S$.  The problem is to find the point $\boldsymbol{p} \in \cal S$ that minimizes the Euclidian 2-norm of the error $\boldsymbol{e} = \boldsymbol{b} - \boldsymbol{p}$ where $||\boldsymbol{e}||_2 = \sqrt{\boldsymbol{e}^T\boldsymbol{e}}$

Inspection of the figure shows geometrically that the shortest error will be to find the point $\boldsymbol{p} = x_1\boldsymbol{a}_1 + x_2\boldsymbol{a}_2$ such that $\boldsymbol{e}$ is orthogonal to the plane $\cal S$.  If we define a matrix 

$$
A = \begin{bmatrix}
    \boldsymbol{a}_1 & \boldsymbol{a}_2
    \end{bmatrix}
$$

whose columns are the vectors $\boldsymbol{a}_1$ and $\boldsymbol{a}_2$, then $\boldsymbol{p} = A\boldsymbol{x}$ where 

$$
\boldsymbol{x} = \begin{bmatrix}
    x_1\\
    x_2
    \end{bmatrix}
$$

Now for $\boldsymbol{e}$ to be orthogonal to every vector in $\cal S$ it is sufficient for it to orthogonal to the basis vectors $\boldsymbol{a}_1$ and $\boldsymbol{a}_2$.  Thus the requirement of orthogonality is that 

$$
    \boldsymbol{a}_i^T\boldsymbol{e} = 0
$$

for $i=1,2$.  This is equivalent to the statement $A^T\boldsymbol{e}=\boldsymbol{0}$  (or that the error is in the Left null space of $A$).

But
\begin{align}
    A^T\boldsymbol{e} & = A^T(\boldsymbol{b} - \boldsymbol{p}) \\
                      & = A^T(\boldsymbol{b} - A\boldsymbol{x}) \\
                      & = \boldsymbol{0}
\end{align}

or the Least-squares solution requires solving the Normal Equations

$$
    A^TA\boldsymbol{x} = A^T\boldsymbol{b}
$$

(although the numerically sensible  way to solve this is to actually use the QR factorization which is another application of orthogonal projection)

Given a solution $\boldsymbol{x}$, the projection is simply $\boldsymbol{p} = A\boldsymbol{x}$

### Orthogonal Projection of a function onto a discrete Function Space

If you understand the previous problem from linear algebra,  you'll realize that orthogonal projection of a function onto a discrete function space is essentially the same problem with slightly different machinery.  

The problem can be stated:

Given a continuous function $f(x)$ and discrete function space $\cal V_h$,  find the function $P_h f\in \cal V_h$ that minimizes $ || f - P_h f||_{L^2}$.  

Again, we can construct the problem geometrically.  First if $P_h f \in \cal V_h$ then we can write it in terms of the basis functions of $\cal V_h$ i.e.

$$
    P_h f(x)  = \sum_{j=0}^N w_j\phi_j(x)
$$

Next we construct the residual (or error) as

$$ 
    r(x) = f(x) - P_h f(x) 
$$
and require that the residual is "orthogonal" to every function $v_h\in \cal V_h$ where we define orthogonality of functions with respect to the $L^2$ norm in the usual way.  i.e. we say that $f$ and $g$ are orthogonal functions if

$$
    \int_\Omega fg dx = 0
$$

Therefore for $r$ to be orthogonal to every function $v_h\in \cal V_h$, it is only required that it be orthogonal to every basis function of $\cal V_h$.  For a finite dimensional function space the problem becomes find $P_h f \in \cal V_h$ such that

$$ 
    \int_\Omega \phi_i(x) r(x) dx = 0
$$ 
for all $i=0,\ldots,N$.  

Using our definitions for $r(x)$ and $P_h f(x)$, the problem becomes,  find the vector of weights $\boldsymbol{w} = [ w_0, w_1,\ldots, w_N]^T$ such that

$$
    \int_\Omega \phi_i(x)\sum_{j=0}^N w_j\phi_j(x) dx = \int_\Omega \phi_i(x) f(x) dx
$$ 
for all $i=0,\ldots,N$.  

Now the RHS of this equation is a set of $N$ numbers that each describes the $L^2$ inner product of $f$ with each basis function. Defining

$$
    \tilde{f}_i = \int_\Omega \phi_i(x) f(x) dx\quad i=0,\ldots,N
$$
the RHS assembles into a vector $\tilde{\boldsymbol{f}}\in R^{N+1}$.  

The LHS is a bit more complicated but we can interchange summation and integration to rewrite the LHS as

$$
\int_\Omega \phi_i(x)\sum_{j=0}^N w_j\phi_j(x) dx = \sum_{j=0}^N w_j\int_\Omega \phi_i(x)\phi_j(x) dx
$$

Now the final definite integrals are another set of doubly indexed numbers

$$
    M_{ij} = \int_\Omega \phi_i(x)\phi_j(x) dx
$$

which define the entries of the "Mass Matrix" $M$.  

Thus the projection problem actually reduces to a Linear algebra 
$$
    \sum_{j=0}^N M_{ij}w_j = \tilde{f}_i
$$
or in matrix vector notation,  Solve
$$
    M\boldsymbol{w} = \tilde{\boldsymbol{f}}
$$
for $\boldsymbol{w}$ then $P_h f(x)$ is defined by the weights and basis functions.

![](./images/FEM_projection_assembly.png)

Figure 9 showing all the components needed to assemble the mass matrix $M$ and the load vector $\tilde{\boldsymbol{f}}$

### Structure of the Mass Matrix

To understand the structure of the mass matrix, it is useful to examine the above figure.

Given a 1-D interval with $N$ elements,  the Mass Matrix $M$ is a $N+1\times N+1$ matrix who's components are given by 

$$
    M_{ij} = \int_\Omega \phi_i(x)\phi_j(x) dx
$$

If the basis functions were Orthonormal (e.g. sines and cosines or Chebyshev polynomials) the Mass Matrix would be equal to the Identity $I$.  However,  if the basis functions are hat functions examination of figure 9 shows that it should be sparse as Basis functions that don't share the same support should have $M_{ij}=0$.  For the case of 1-D problems in $P_1$,  it should be clear that the Mass Matrix should actually be tri-diagonal  as the only non-zero elements will arise from the interaction of a basis function $\phi_i$ and it's two nearest neighbors $\phi_{i-1}$ and $\phi_{i+1}$

In particular,  the $i$th row of $M$ will have three non-zero entries

\begin{align}
    M_{i,i-1} &= \int_\Omega \phi_{i}\phi_{i-1} dx = \int_{x_{i-1}}^{x_i} \phi_{i}\phi_{i-1} dx \\
    M_{i,i} &= \int_\Omega \phi_{i}\phi_{i} dx = \int_{x_{i-1}}^{x_{i+1}} \phi_{i}\phi_{i} dx \\
    M_{i,i+1} &= \int_\Omega \phi_{i}\phi_{i+1} dx = \int_{x_i}^{x_{i+1}} \phi_{i}\phi_{i+1} dx \\
\end{align}

Note that the first term is contributed entirely by element $e_{i-1}$, the last term by element $e_i$ and the diagonal entry actually can be split into two parts

$$
    M_{i,i} = \int_{x_{i-1}}^{x_{i}} \phi_{i}\phi_{i} dx + \int_{x_{i}}^{x_{i+1}} \phi_{i}\phi_{i} dx
$$

corresponding to the contributions from both elements $e_{i-1}$ and $e_i$.  For simple  $P_1$ hat functions we can actually calculate these entries analytically (although for more complicated problems or the load matrix we might want to use numerical quadrature).  

If we just consider the first term and use the definitions of $\phi_{i-1}$ and $\phi_i$

$$
    M_{i,i-1} = \int_{x_{i-1}}^{x_i} \frac{x_{i} - x}{h_{i-1}} \frac{x - x_{i-1}}{h_{i-1}} dx 
$$

which is straightforward to integrate,  but is easier to do if we first make the affine transformation

$$
 x(t) = x_{i-1} + t h_{i-1}
$$

where again $t\in [0,1]$ is the fractional distance across element $e{i-1}$ ($t$ is also known as the local element coordinate).  Given this transformation and $dx = h_{i-1} dt$,  the above integral simplifies to

$$
        M_{i,i-1} = h_{i-1}\int_{0}^{1} ( 1- t) t dt = h_{i-1} \left.\left( \frac{t^2}{2} - \frac{t^3}{3}\right)\right|_0^1 = \frac{h_{i-1}}{6}
$$ 

Using the same approach we can evaluate the remaining terms
\begin{align}
M_{i,i} &= h_{i-1}\int_{0}^{1} t^2 dt  + h_i\int_{0}^{1} (1 -t)^2 dt = \frac{h_{i-1}}{3} + \frac{h_i}{3}\\

M_{i,i+1} & = h_i\int_{0}^{1} t(1-t) dt = \frac{h_i}{6}
\end{align}

so the stencil for one row of the mass matrix is simply

$$
M_i = \begin{bmatrix}
   \frac{h_{i-1}}{6}  & \frac{h_{i-1}}{3} + \frac{h_i}{3} & \frac{h_i}{6}
    \end{bmatrix}
$$

and the Global Mass matrix looks like
$$
    M = \begin{bmatrix}
    \frac{h_0}{3}  & \frac{h_0}{6} & & & & \\
    \frac{h_0}{6}  & \frac{h_0}{3} + \frac{h_1}{3} & \frac{h_1}{6} & & & \\
    &\frac{h_1}{6}  & \frac{h_1}{3} + \frac{h_2}{3} & \frac{h_2}{6}  & & \\
    & & \ddots & \ddots & \ddots & \\
    & & &   &  \frac{h_N}{6} & \frac{h_N}{3}
    \end{bmatrix}
$$

### Mechanics:  Element Wise Assembly 

While it is possible to assemble the global mass matrix as described above,  a better approach is to loop over each element and calculate the local contribution of each element to the global mass matrix.  To begin with we can decompose $M$ into a sum of simpler block matrices

$$
M = \begin{bmatrix}
\frac{h_0}{3}  & \frac{h_0}{6} & &  \\
\frac{h_0}{6}  & \frac{h_0}{3}  & & \\ 
&&& \\
&&& \\
&&& \\
&&& 
\end{bmatrix} +
\begin{bmatrix}
&&& \\
& \frac{h_1}{3}  & \frac{h_1}{6}   & \\ 
& \frac{h_1}{6}  & \frac{h_1}{3}  &  \\
&&& \\
&&& \\
&&& 
\end{bmatrix} + \ldots +
\begin{bmatrix}
&&& \\
&&& \\
&&& \\
&&& \\
& & \frac{h_N}{3}  & \frac{h_N}{6} \\
& &\frac{h_N}{6}  & \frac{h_N}{3}  
\end{bmatrix} 
$$

or

$$
M = M^{e_0} + M^{e_1} + \ldots + M^{e_N}
$$

where 

$$
  M^{e_i} = \frac{h_i}{6}\begin{bmatrix}
  2 & 1\\
  1 & 2
  \end{bmatrix}
  $$
  
  is the local element Mass Matrix for element $e_i$.  Examination of Figure 3 shows that we can describe the element mass matrix entirely in terms of the local basis functions $N_0(t)$ and $N_1(t)$ in the local coordinate system as
  
$$
  M^{e_i} = h_i\begin{bmatrix}
  \int_0^1 N_0^2 dt & \int_0^1 N_0N_1 dt \\
  \int_0^1 N_0N_1 dt & \int_0^1 N_1^2 dt 
\end{bmatrix}
$$

and each of these entries correspond to parts (or all) of entries in the global mass matrix

$$
  M^{e_i} = \begin{bmatrix}
  M_{i,i} & M_{i,i+1} \\
  M_{i+1,i} & M_{i+1,i+1} 
\end{bmatrix}
$$

So the overall algorithm is to 
* Loop over elements $i=0,\ldots,N$
* Calculate the local Mass Matrix $M^{e_i}$
* Insert/add into the Global Mass Matrix

A similar process is used to assemble the load vector $\tilde{\boldsymbol{f}}$ by 
* Loop over elements $i=0,\ldots,N$
* Calculate the element contribution to the load vector

$$
    \tilde{\boldsymbol{f}}^{e_i} = \begin{bmatrix}
        \int_{x_{i-1}}^{x_i} N_0(x)f(x) dx \\
        \int_{x_{i-1}}^{x_i} N_1(x)f(x) dx 
        \end{bmatrix} = \begin{bmatrix}
        \tilde{f}_i \\
        \tilde{f}_{i+1} 
        \end{bmatrix}
$$
* Insert/Add into the Global Load Vector



### Convergence/Optimality ?